In [ ]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import easyocr
import csv
import uuid

In [ ]:
#set paths and category index

PATH_TO_SAVED_MODEL = "training_op/models/saved_model/"
PATH_TO_LABELS = "training_op/annotations/label_map.pbtxt"

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [ ]:

#gpu configurations edit (IF REQUIRED)

def gpu_config(memory=4120):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(gpus)
        try:
            tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory)])
            print('success')
        except RuntimeError as e:
            print(e)

gpu_config(memory=3120)


In [ ]:
#OCR Functions

#filter number from numberplate
def filter_text(region, ocr_result, region_threshold=0.3):
    rectangle_size = region.shape[0] * region.shape[1]
    plate = []

    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        if length * height / rectangle_size > region_threshold:
            plate.append(result[1])

    return plate

In [ ]:
#apply ocr
def ocr_it(image, detections, detection_threshold=0.8, region_threshold=0.3):
    scores = list(filter(lambda x: x > detection_threshold, detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]

    width = image.shape[1]
    height = image.shape[0]

    if len(boxes) != 0:
        for box in boxes:
            roi = box * [height, width, height, width]
            region = image[int(roi[0]):int(roi[2]), int(roi[1]):int(roi[3])]
            reader = easyocr.Reader(['en'], gpu=True, verbose=False)
            ocr_result = reader.readtext(region)
            text = filter_text(region, ocr_result, region_threshold)
            # plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
            return text, region

In [ ]:
#save results function
def save_results(text, region, csv_filename, folder_path):
    img_name = f'{uuid.uuid1()}.jpg'
    cv2.imwrite(os.path.join(folder_path, img_name), region)

    with open(csv_filename, mode="a", newline='') as f:
        csv_writer = csv.writer(f, delimiter=',')
        csv_writer.writerow([img_name, text])


In [ ]:
#load model

detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

In [ ]:
## PREDICT FROM VIDEO


# load frame and do inference
cap = cv2.VideoCapture('trainingop/test_images/videoplayback.mp4') 
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    image_np = np.array(frame)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.uint8)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 0
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=.5,
        agnostic_mode=False)

    try:
        text, region = ocr_it(image_np_with_detections, detections, detection_threshold, region_threshold=0.3)
        # save_results(str(text), region, 'detection_results.csv', 'detection_images')
    except:
        pass

#     print(f"License Plate Number: {text}")
#     plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
#     cv2.putText(image_np_with_detections, str(text), (50, 140), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 255, 0), 8)
    try:
        cv2.imshow('plate', cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
    except:
        pass
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break


In [ ]:
### IF PREDICTING FROM IMAGE


IMAGE_PATH = os.path.join('training_op', 'test_images', 'Cars401.png')

img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.uint8)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 0
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.5,
            agnostic_mode=False)

text, region = ocr_it(image_np_with_detections, detections, detection_threshold, region_threshold=0.3)
# save_results(str(text), region, 'detection_results.csv', 'detection_images')


print(f"License Plate Number: {text}")
plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))

cv2.putText(image_np_with_detections, str(text), (50, 140), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 255, 0), 8)

plt.figure(figsize=(10,7))
plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show() 